<a href="https://colab.research.google.com/github/oijiin/ANALISE_DE_DADOS/blob/main/SIMULADOR_TMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proposta de Código Inicial (Simulador TMS Básico em Python)

In [31]:
import datetime
import math # Para cálculos de distância (exemplo)
import logging
from enum import Enum # Importar Enum
import time

In [ ]:
# --- Configuração básica do Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# --- Enum para Status da Carga ---
class CargaStatus(Enum):
    CRIADA = "CRIADA"
    PLANEJADA = "PLANEJADA"
    DESPACHADA = "DESPACHADA" # Renomeado/Adicionado para clareza
    EM_TRANSITO = "EM_TRANSITO"
    ENTREGA_FALHOU = "ENTREGA_FALHOU" # Status adicional
    ENTREGUE = "ENTREGUE"
    CANCELADA = "CANCELADA"

In [ ]:
class Endereco:
    """
        Representa um endereço geográfico simples.
    """

    def __init__(self, logradouro, numero, cidade, estado, cep, pais="Brasil"):
        """
          Inicializa um endereço com os atributos essenciais.
        """
        # Validações básicas podem ser adicionadas aqui
        if not all([logradouro, cidade, estado, cep]): # Validação básica
            raise ValueError("Logradouro, cidade, estado e CEP são obrigatórios.")

        self.logradouro = logradouro
        self.numero = numero
        self.cidade = cidade
        self.estado = estado
        self.cep = cep
        self.pais = pais
        # Poderíamos adicionar latitude/longitude para cálculos de distância reais
        # self.latitude = latitude
        # self.longitude = longitude

    def __str__(self):
        """
          Formata o endereço para exibição.
        """
        num_str = f", {self.numero}" if self.numero else ""
        return f"{self.logradouro}{num_str} - {self.cidade}/{self.estado} - CEP: {self.cep}"

    def __repr__(self):
        """
          Representação em formato de string para fins de depuração.
        """
        return (f"Endereco(logradouro='{self.logradouro}', numero='{self.numero}', "
                f"cidade='{self.cidade}', estado='{self.estado}', cep='{self.cep}')")

In [ ]:
class ItemTransporte:
    """
      Representa um item específico dentro de uma carga, com suas dimensões logísticas.
    """
    def __init__(self, sku, quantidade, peso_unitario_kg, volume_unitario_m3):
        """
          Inicializa um item com SKU, quantidade, peso e volume.
        """
        if quantidade <= 0 or peso_unitario_kg <= 0 or volume_unitario_m3 <= 0:
            raise ValueError("Quantidade, peso e volume devem ser positivos.")
        self.sku = sku
        self.quantidade = quantidade
        self.peso_unitario_kg = peso_unitario_kg
        self.volume_unitario_m3 = volume_unitario_m3

    @property
    def peso_total_kg(self):
        """Calcula o peso total do item em quilogramas."""
        return self.quantidade * self.peso_unitario_kg

    @property
    def volume_total_m3(self):
        """Calcula o volume total do item em metros cúbicos."""
        return self.quantidade * self.volume_unitario_m3

    def __str__(self):
        """Formata o item para exibição."""
        return (f"Item(SKU: {self.sku}, Qtd: {self.quantidade}, "
                f"Peso Total: {self.peso_total_kg:.2f} kg, "
                f"Vol Total: {self.volume_total_m3:.4f} m³)")

    def __repr__(self):
         """Representação em formato de string para fins de depuração."""
         return (f"ItemTransporte(sku='{self.sku}', quantidade={self.quantidade}, "
                 f"peso_unitario_kg={self.peso_unitario_kg}, "
                 f"volume_unitario_m3={self.volume_unitario_m3})")

In [25]:
class Carga:
    """
      Representa uma carga a ser transportada (um conjunto de itens de um WMS, por exemplo).
    """

    def __init__(self, id_carga: str, origem: Endereco, destino: Endereco, itens: list[ItemTransporte]):
        if not id_carga or not isinstance(origem, Endereco) or not isinstance(destino, Endereco):
             raise ValueError("ID da carga, origem e destino são obrigatórios e devem ser dos tipos corretos.")
        if not itens:
            raise ValueError("A carga deve conter pelo menos um item.")

        self.id_carga = id_carga
        self.origem = origem
        self.destino = destino
        self.itens = itens # Lista de objetos ItemTransporte
        self.status = CargaStatus.CRIADA # Usando Enum
        self.data_criacao = datetime.datetime.now()
        self.data_prevista_entrega = None
        self.data_efetiva_entrega = None # Adicionado
        self.transportadora_designada = None
        self.custo_frete_calculado = None # Renomeado para clareza
        self.tracking_history = [(self.data_criacao, self.status, "Carga criada no sistema.")]
        logging.info(f"Carga {self.id_carga} criada: {self.origem.cidade} -> {self.destino.cidade}")

    @property
    def peso_total_carga_kg(self) -> float:
        return sum(item.peso_total_kg for item in self.itens)

    @property
    def volume_total_carga_m3(self):
        return sum(item.volume_total_m3 for item in self.itens)

    def atualizar_status(self, novo_status: CargaStatus, detalhes: str, transportadora=None, custo: float = None, data_prevista: datetime.datetime = None):
        """Atualiza o status da carga e registra no histórico."""
        if not isinstance(novo_status, CargaStatus):
            logging.warning(f"Tentativa de atualizar Carga {self.id_carga} com status inválido: {novo_status}")
            return

        timestamp = datetime.datetime.now()
        self.status = novo_status
        self.tracking_history.append((timestamp, self.status, detalhes))

        if transportadora: self.transportadora_designada = transportadora
        if custo is not None: self.custo_frete_calculado = custo
        if data_prevista: self.data_prevista_entrega = data_prevista
        if novo_status == CargaStatus.ENTREGUE:
             self.data_efetiva_entrega = timestamp

        logging.info(f"Carga {self.id_carga}: Status -> {self.status.value}. Detalhes: {detalhes}")

    def __str__(self):
        itens_str = "\n    ".join(map(str, self.itens))
        hist_str = "\n    ".join([f"{ts.strftime('%Y-%m-%d %H:%M:%S')} - {st.value} - {det}" for ts, st, det in self.tracking_history])
        custo_str = f"R$ {self.custo_frete_calculado:.2f}" if self.custo_frete_calculado is not None else "Não calculado"
        prev_entrega_str = self.data_prevista_entrega.strftime('%Y-%m-%d') if self.data_prevista_entrega else "Não definida"
        efet_entrega_str = self.data_efetiva_entrega.strftime('%Y-%m-%d %H:%M') if self.data_efetiva_entrega else "Pendente"
        transportadora_nome = self.transportadora_designada.nome if self.transportadora_designada else 'Não designada'

        return (f"--- Carga ID: {self.id_carga} | Status: {self.status.value} ---\n"
                f"  Origem: {self.origem}\n"
                f"  Destino: {self.destino}\n"
                f"  Peso Total: {self.peso_total_carga_kg:.2f} kg\n"
                f"  Volume Total: {self.volume_total_carga_m3:.4f} m³\n"
                f"  Transportadora: {transportadora_nome}\n"
                f"  Custo Calculado: {custo_str}\n"
                f"  Previsão Entrega: {prev_entrega_str}\n"
                f"  Entrega Efetiva: {efet_entrega_str}\n"
                f"  Itens:\n    {itens_str}\n"
                f"  Histórico:\n    {hist_str}")

    def __repr__(self):
        """Representação em formato de string para fins de depuração."""
        return (f"Carga(id_carga='{self.id_carga}', status={self.status}, "
                f"origem={self.origem!r}, destino={self.destino!r})")

In [29]:
class Transportadora:
    """Representa uma empresa de transporte."""
    def __init__(self, id_transportadora, nome, taxa_km_kg=0.10, taxa_km_m3=5.0, taxa_fixa=50.0, cap_max_kg=10000.0, cap_max_m3=50.0):
        """
        Inicializa a transportadora.
        taxa_km_kg: Custo por quilômetro por quilograma.
        taxa_km_m3: Custo por quilômetro por metro cúbico (peso cubado).
        taxa_fixa: Custo fixo por viagem.
        cap_max_kg/m3: Capacidade máxima *simulada* que a transportadora consegue lidar numa única carga/viagem.
        """
        self.id_transportadora = id_transportadora
        self.nome = nome
        self.taxa_km_kg = taxa_km_kg
        self.taxa_km_m3 = taxa_km_m3
        self.taxa_fixa = taxa_fixa
        self.capacidade_max_kg = cap_max_kg
        self.capacidade_max_m3 = cap_max_m3
        # Poderia ter: frota de veículos, áreas de atendimento, tempo médio, etc.

    def pode_transportar(self, carga: Carga) -> bool:
        """
        Verifica de forma simplista se a transportadora tem capacidade para a carga.

        """
        return (carga.peso_total_carga_kg <= self.capacidade_max_kg and
                carga.volume_total_carga_m3 <= self.capacidade_max_m3)

    def calcular_custo_frete(self, carga: Carga, distancia_km: float) -> float:
        """
        Calcula um custo de frete um pouco mais elaborado.
        """
        if not self.pode_transportar(carga):
            raise ValueError("Capacidade insuficiente para a carga.")

        if distancia_km <= 0:
            raise ValueError("Distância deve ser positiva.")

        custo_peso = distancia_km * self.taxa_km_kg * carga.peso_total_carga_kg
        custo_volume = distancia_km * self.taxa_km_m3 * carga.volume_total_carga_m3

        # Custo baseado no maior entre peso e volume (simulando peso cubado) + taxa fixa
        custo_variavel = max(custo_peso, custo_volume)
        custo_total = custo_variavel + self.taxa_fixa

        return custo_total

    def __str__(self):
        """
        Formata a transportadora para exibição.
        """
        return f"Transportadora(ID: {self.id_transportadora}, Nome: {self.nome})"

    def __repr__(self):
        """
        Representação em formato de string para fins de depuração.
        """
        return (f"Transportadora(id='{self.id_transportadora}', nome='{self.nome}', "
                f"taxa_kg={self.taxa_km_kg}, taxa_m3={self.taxa_km_m3}, fixa={self.taxa_fixa})")

In [28]:
class TMS:
    """
    Sistema de Gerenciamento de Transporte - Orquestrador.

    Simula um sistema de gerenciamento de transporte simples.
    """
    def __init__(self):
        """
        Inicializa o sistema com dicionários vazios para cargas e transportadoras.
        """
        self.cargas: dict[str, Carga] = {}
        self.transportadoras: dict[str, Transportadora] = {}
        # Mapa de distâncias simuladas (poderia ser carregado de um arquivo)
        self.distancias_simuladas = {
            ("São Paulo", "Rio de Janeiro"): 450, ("Rio de Janeiro", "São Paulo"): 450,
            ("São Paulo", "Campinas"): 95, ("Campinas", "São Paulo"): 95,
            ("São Paulo", "Belo Horizonte"): 586, ("Belo Horizonte", "São Paulo"): 586,
            ("Rio de Janeiro", "Belo Horizonte"): 434, ("Belo Horizonte", "Rio de Janeiro"): 434,
        }
        # Simulação de um catálogo de produtos para buscar peso/volume
        self.catalogo_produtos = {
            "SKU001": {"desc": "Parafuso Sextavado 1/4", "peso_kg": 0.02, "vol_m3": 0.00001},
            "SKU002": {"desc": "Porca Sextavada 1/4", "peso_kg": 0.01, "vol_m3": 0.000008},
            "SKU003": {"desc": "Monitor LED 24 pol", "peso_kg": 4.5, "vol_m3": 0.05},
            "SKU004": {"desc": "Teclado Gamer RGB", "peso_kg": 0.8, "vol_m3": 0.008},
            "SKU005": {"desc": "Pallet de Cimento (50 sacos)", "peso_kg": 2500, "vol_m3": 1.5}, # Item pesado/volumoso
        }

    def adicionar_transportadora(self, transportadora: Transportadora):
        """
        Adiciona uma transportadora ao sistema.

        Verifica se a transportadora é do tipo Transportadora antes de adicionar.

        Retorna True se a transportadora for adicionada com sucesso, False caso contrário.
        """
        if not isinstance(transportadora, Transportadora):
            raise TypeError("Objeto não é do tipo Transportadora.")
        if transportadora.id_transportadora in self.transportadoras:
            logging.warning(f"Transportadora {transportadora.id_transportadora} ('{transportadora.nome}') já cadastrada.")
            return False
        self.transportadoras[transportadora.id_transportadora] = transportadora
        logging.info(f"Transportadora '{transportadora.nome}' (ID: {transportadora.id_transportadora}) adicionada.")
        return True

    def _simular_distancia_km(self, origem: Endereco, destino: Endereco) -> float:
        """
        Simula o cálculo de distância.

        Retorna a distância em quilômetros.
        """
        par_cidades = (origem.cidade, destino.cidade)
        dist = self.distancias_simuladas.get(par_cidades)
        if dist is None:
            # Tenta a ordem inversa
            par_cidades_inverso = (destino.cidade, origem.cidade)
            dist = self.distancias_simuladas.get(par_cidades_inverso)

        if dist is None:
            logging.warning(f"Distância não encontrada entre {origem.cidade} e {destino.cidade}. Usando valor padrão 100km.")
            # Cálculo simples de fallback baseado em "distância aérea" se tivéssemos lat/lon
            # ou um valor padrão fixo.
            return 100.0
        return float(dist)

    def _criar_item_transporte(self, sku: str, quantidade: int) -> ItemTransporte | None:
        """
        Busca dados do item no catálogo e cria o objeto ItemTransporte.
        Retorna None se o SKU não for encontrado.
        """
        if sku not in self.catalogo_produtos:
            logging.error(f"SKU '{sku}' não encontrado no catálogo.")
            return None

        dados_item = self.catalogo_produtos.get(sku)

        if not dados_item:
            logging.error(f"SKU '{sku}' não encontrado no catálogo.")
            return None
        try:
            return ItemTransporte(
                sku=sku,
                quantidade=quantidade,
                peso_unitario_kg=dados_item['peso_kg'],
                volume_unitario_m3=dados_item['vol_m3']
            )
        except ValueError as e:
            logging.error(f"Erro ao criar ItemTransporte para SKU {sku}: {e}")
            return None

    def criar_carga_por_skus(self, id_carga: str, origem: Endereco, destino: Endereco, itens_sku_qtd: list[tuple[str, int]]) -> Carga | None:
        """
        Cria uma nova carga buscando os detalhes dos itens pelo SKU.

        Retorna a carga criada ou None em caso de erro.
        """
        if not isinstance(origem, Endereco) or not isinstance(destino, Endereco):
            logging.error("Origem e destino devem ser instâncias de Endereco.")
            return None

        if not id_carga:
            logging.error("ID da carga é obrigatório.")
            return None

        if id_carga in self.cargas:
            logging.error(f"Carga com ID {id_carga} já existe.")
            return None

        if not itens_sku_qtd:
             logging.error(f"Nenhum item (SKU, quantidade) fornecido para a carga {id_carga}.")
             return None

        itens_transporte = []

        for sku, qtd in itens_sku_qtd:
            item = self._criar_item_transporte(sku, qtd)
            if item:
                itens_transporte.append(item)
            else:
                # Se um item falhar, talvez a carga inteira não deva ser criada? Decisão de negócio.
                logging.error(f"Falha ao criar item para SKU {sku}. Carga {id_carga} não será criada.")
                return None # Ou poderia continuar e criar a carga com os itens válidos

        if not itens_transporte:
            logging.error(f"Nenhum item válido pôde ser criado para a carga {id_carga}.")
            return None

        try:
            nova_carga = Carga(id_carga, origem, destino, itens_transporte)
            self.cargas[id_carga] = nova_carga
            # Logging já é feito dentro de Carga.__init__
            return nova_carga
        except ValueError as e:
            logging.error(f"Erro ao criar carga {id_carga}: {e}")
            return None

    def planejar_transporte(self, id_carga: str) -> bool:
        """
        Seleciona a transportadora (mais barata que pode transportar) e calcula o custo.
        """
        carga = self.cargas.get(id_carga)

        if not carga:
            logging.error(f"Carga {id_carga} não encontrada para planejamento.")
            return False

        if carga.status != CargaStatus.CRIADA:
            logging.error(f"Carga {id_carga} não está no status CRIADA (status: {carga.status.value}). Planejamento cancelado.")
            return False

        if not self.transportadoras:
            logging.error("Nenhuma transportadora cadastrada para planejar o transporte.")
            return False

        distancia_km = self._simular_distancia_km(carga.origem, carga.destino)
        logging.info(f"Iniciando planejamento para Carga {id_carga} ({carga.origem.cidade} -> {carga.destino.cidade}, {distancia_km} km)")
        logging.info(f"Peso Carga: {carga.peso_total_carga_kg:.2f} kg, Volume Carga: {carga.volume_total_carga_m3:.4f} m³")


        melhor_transportadora = None
        menor_custo = float('inf')
        transportadoras_candidatas = []

        for transportadora in self.transportadoras.values():
            logging.debug(f"Avaliando Transportadora: {transportadora.nome} (Cap: {transportadora.capacidade_max_kg}kg / {transportadora.capacidade_max_m3}m³)")
            # 1. Verifica Capacidade
            if not transportadora.pode_transportar(carga):
                logging.warning(f"Transportadora {transportadora.nome} não tem capacidade para a carga {id_carga}.")
                continue

            # 2. Calcula Custo
            try:
                custo_estimado = transportadora.calcular_custo_frete(carga, distancia_km)
                transportadoras_candidatas.append((transportadora, custo_estimado))
                logging.info(f"  -> {transportadora.nome}: Custo Calculado R$ {custo_estimado:.2f}")
                if custo_estimado < menor_custo:
                    menor_custo = custo_estimado
                    melhor_transportadora = transportadora
            except ValueError as e:
                logging.error(f"Erro ao calcular custo para {transportadora.nome} na carga {id_carga}: {e}")

        if melhor_transportadora:
            # Simulação simples de tempo de trânsito (ex: 1 dia fixo + 1 dia a cada 400km)
            dias_transporte = 1 + math.ceil(distancia_km / 400)
            data_prevista = carga.data_criacao + datetime.timedelta(days=dias_transporte)

            detalhes = (f"Planejamento concluído. Transportadora: {melhor_transportadora.nome}. "
                        f"Custo: R$ {menor_custo:.2f}. Prev. Entrega: {data_prevista.strftime('%Y-%m-%d')}")
            carga.atualizar_status(CargaStatus.PLANEJADA, detalhes,
                                   transportadora=melhor_transportadora,
                                   custo=menor_custo,
                                   data_prevista=data_prevista)
            return True
        else:
            logging.error(f"Não foi encontrada transportadora com capacidade/disponível para a carga {id_carga}.")
            # Poderia ter um status "FALHA_PLANEJAMENTO"
            carga.atualizar_status(CargaStatus.CRIADA, "Falha no planejamento: Nenhuma transportadora adequada encontrada.") # Volta ou mantém CRIADA?
            return False

    def despachar_carga(self, id_carga: str) -> bool:
        """
        Muda o status da carga para DESPACHADA (coletada).

        Simula a carga entrando em trânsito.
        """
        carga = self.cargas.get(id_carga)

        if not carga:
            logging.error(f"Carga {id_carga} não encontrada para despacho.")
            return False
        if carga.status != CargaStatus.PLANEJADA:
             logging.error(f"Carga {id_carga} precisa estar PLANEJADA para ser despachada (status: {carga.status.value}).")
             return False
        if not carga.transportadora_designada:
             logging.error(f"Carga {id_carga} está PLANEJADA mas sem transportadora designada.")
             return False # Ou tentar replanejar?

        detalhes = f"Carga coletada pela transportadora {carga.transportadora_designada.nome}."
        carga.atualizar_status(CargaStatus.DESPACHADA, detalhes)
        # Aqui poderia disparar a próxima atualização para EM_TRANSITO após um tempo simulado
        self.simular_inicio_transito(id_carga, delay_seconds=5) # Exemplo de simulação
        return True

    def simular_inicio_transito(self, id_carga: str, delay_seconds: int = 0):
        """
        Simula a carga entrando em trânsito após um tempo.

        Pode ser implementado com threading.Timer ou asyncio.sleep.

        Delay é em segundos.
        """
        # Em um sistema real, isso seria acionado por um evento externo (confirmação da transportadora)

        # Aqui, apenas atualizamos o status. O 'delay' seria implementado com threading.Timer ou asyncio.sleep

        # Para simplificar, vamos chamar diretamente, mas a ideia está aqui.
        import time

        time.sleep(delay_seconds) # *** NÂO FAÇA ISSO EM CÓDIGO ASSÍNCRONO REAL ***

        carga = self.cargas.get(id_carga)

        if not carga: return # Carga pode ter sido cancelada, etc.

        if carga.status == CargaStatus.DESPACHADA:
             detalhes = "Veículo iniciou o trajeto."
             carga.atualizar_status(CargaStatus.EM_TRANSITO, detalhes)

    def registrar_evento_transito(self, id_carga: str, descricao_evento: str):
        """
        Registra um evento intermediário durante o trânsito.
        """
        carga = self.cargas.get(id_carga)
        if not carga:
            logging.error(f"Carga {id_carga} não encontrada para registrar evento.")
            return False
        if carga.status not in [CargaStatus.DESPACHADA, CargaStatus.EM_TRANSITO]:
             logging.warning(f"Carga {id_carga} não está em trânsito para registrar evento (status: {carga.status.value}).")
             # Poderia registrar mesmo assim, dependendo da regra de negócio
             return False

        # Mantém o status EM_TRANSITO, mas adiciona ao histórico
        timestamp = datetime.datetime.now()
        carga.tracking_history.append((timestamp, carga.status, descricao_evento))
        logging.info(f"Carga {id_carga}: Evento -> {descricao_evento}")
        return True

    def registrar_falha_entrega(self, id_carga: str, motivo: str):
        """
        Registra uma tentativa de entrega que falhou.
        """
        carga = self.cargas.get(id_carga)
        if not carga:
            logging.error(f"Carga {id_carga} não encontrada para registrar falha na entrega.")
            return False
        if carga.status != CargaStatus.EM_TRANSITO:
             logging.error(f"Carga {id_carga} precisa estar EM_TRANSITO para registrar falha na entrega (status: {carga.status.value}).")
             return False

        detalhes = f"Tentativa de entrega falhou. Motivo: {motivo}"
        carga.atualizar_status(CargaStatus.ENTREGA_FALHOU, detalhes)
        # Aqui, o processo pode precisar de intervenção manual ou nova tentativa automática
        return True

    def registrar_entrega(self, id_carga: str):
         """
         Muda o status da carga para ENTREGUE.

         Simula a entrega confirmada.

         Pode ser implementado com um evento externo.
         """
         carga = self.cargas.get(id_carga)
         if not carga:
             logging.error(f"Carga {id_carga} não encontrada para registrar entrega.")
             return False
         # Permite registrar entrega mesmo se houve falha anterior (nova tentativa)
         if carga.status not in [CargaStatus.EM_TRANSITO, CargaStatus.ENTREGA_FALHOU]:
              logging.error(f"Carga {id_carga} não está em trânsito ou com falha para ser marcada como entregue (status: {carga.status.value}).")
              return False

         detalhes = f"Entrega confirmada no destino ({carga.destino.cidade})."
         carga.atualizar_status(CargaStatus.ENTREGUE, detalhes)
         return True

    def cancelar_carga(self, id_carga: str, motivo: str):
        """
        Cancela uma carga que ainda não foi entregue.
        """
        carga = self.cargas.get(id_carga)
        if not carga:
            logging.error(f"Carga {id_carga} não encontrada para cancelamento.")
            return False
        if carga.status in [CargaStatus.ENTREGUE]:
            logging.error(f"Carga {id_carga} já foi entregue e não pode ser cancelada.")
            return False
        if carga.status == CargaStatus.CANCELADA:
            logging.warning(f"Carga {id_carga} já está cancelada.")
            return False

        detalhes = f"Carga cancelada. Motivo: {motivo}"
        carga.atualizar_status(CargaStatus.CANCELADA, detalhes)
        # Aqui poderia ter lógica para notificar transportadora, reverter alocação, etc.
        return True

    def consultar_carga(self, id_carga: str):
        """
        Exibe os detalhes de uma carga específica.
        """
        carga = self.cargas.get(id_carga)
        if carga:
            print("-" * 60)
            print(carga)
            print("-" * 60)
        else:
            logging.error(f"Carga {id_carga} não encontrada para consulta.")

    def listar_cargas(self, status_filtro: CargaStatus | None = None):
        """
        Lista as cargas no sistema, opcionalmente filtrando por status.
        """
        print("\n--- Lista de Cargas ---")
        cargas_filtradas = self.cargas.values()
        if status_filtro:
            if not isinstance(status_filtro, CargaStatus):
                logging.error(f"Status de filtro inválido: {status_filtro}")
                return
            cargas_filtradas = [c for c in self.cargas.values() if c.status == status_filtro]
            print(f"Filtrando por Status: {status_filtro.value}")

        if not cargas_filtradas:
            print("Nenhuma carga encontrada" + (f" com o status {status_filtro.value}." if status_filtro else "."))
            return

        for carga in cargas_filtradas:
             transp_nome = carga.transportadora_designada.nome if carga.transportadora_designada else "N/D"
             custo_str = f"R${carga.custo_frete_calculado:.2f}" if carga.custo_frete_calculado is not None else "N/C"
             print(f"ID: {carga.id_carga} | Status: {carga.status.value:<10} | "
                   f"Orig: {carga.origem.cidade:<15} | Dest: {carga.destino.cidade:<15} | "
                   f"Transp: {transp_nome:<20} | Custo: {custo_str}")
        print("-" * 20)

In [32]:
# --- Exemplo de Uso Aprimorado ---
if __name__ == "__main__":
    # 1. Criar o TMS
    meu_tms = TMS()

    # 2. Adicionar Transportadoras com mais detalhes
    t1 = Transportadora("T001", "Expresso Rápido", taxa_km_kg=0.12, taxa_km_m3=6.0, taxa_fixa=60.0, cap_max_kg=5000, cap_max_m3=30)
    t2 = Transportadora("T002", "Log Cargas Plus", taxa_km_kg=0.10, taxa_km_m3=4.5, taxa_fixa=55.0, cap_max_kg=15000, cap_max_m3=80)
    t3 = Transportadora("T003", "Carga Pesada Transp", taxa_km_kg=0.25, taxa_km_m3=10.0, taxa_fixa=150.0, cap_max_kg=30000, cap_max_m3=100) # Mais cara, alta capacidade
    meu_tms.adicionar_transportadora(t1)
    meu_tms.adicionar_transportadora(t2)
    meu_tms.adicionar_transportadora(t3)

    # 3. Definir Endereços
    end_cd_sp = Endereco("Rua do Armazém Central", "1000", "São Paulo", "SP", "01001-000")
    end_cli_rj = Endereco("Avenida Atlântica", "1702", "Rio de Janeiro", "RJ", "22021-001")
    end_cli_cp = Endereco("Rua Barão de Jaguara", "999", "Campinas", "SP", "13015-001")
    end_cli_bh = Endereco("Avenida Afonso Pena", "1500", "Belo Horizonte", "MG", "30130-005")

    # 4. Criar Cargas usando SKUs do "catálogo"
    carga_rj = meu_tms.criar_carga_por_skus(
        "CARGA101", end_cd_sp, end_cli_rj,
        [("SKU003", 2), ("SKU004", 5)] # 2 Monitores, 5 Teclados
    )
    carga_cp = meu_tms.criar_carga_por_skus(
        "CARGA102", end_cd_sp, end_cli_cp,
        [("SKU001", 500), ("SKU002", 1000)] # Muitos parafusos e porcas
    )
    carga_bh_pesada = meu_tms.criar_carga_por_skus(
        "CARGA103", end_cd_sp, end_cli_bh,
        [("SKU005", 1)] # 1 Pallet de cimento - deve exigir T003
    )
    carga_invalida = meu_tms.criar_carga_por_skus(
        "CARGA104", end_cd_sp, end_cli_rj,
        [("SKU_NAO_EXISTE", 1)] # SKU inválido
    )

    meu_tms.listar_cargas()

    # 5. Planejar o Transporte
    if carga_rj: meu_tms.planejar_transporte(carga_rj.id_carga)
    if carga_cp: meu_tms.planejar_transporte(carga_cp.id_carga)
    if carga_bh_pesada: meu_tms.planejar_transporte(carga_bh_pesada.id_carga)

    # Consultar após planejamento
    if carga_rj: meu_tms.consultar_carga(carga_rj.id_carga)
    if carga_bh_pesada: meu_tms.consultar_carga(carga_bh_pesada.id_carga)

    meu_tms.listar_cargas(status_filtro=CargaStatus.PLANEJADA)

    # 6. Simular Despacho -> Trânsito -> Eventos -> Entrega/Falha
    if carga_rj:
        meu_tms.despachar_carga(carga_rj.id_carga) # Atualiza para DESPACHADA e agenda EM_TRANSITO
        # Simulando passagem de tempo e evento
        time.sleep(1) # Pequeno delay para visualização
        meu_tms.registrar_evento_transito(carga_rj.id_carga, "Passagem pelo Posto Fiscal XYZ")
        time.sleep(1)
        # meu_tms.registrar_falha_entrega(carga_rj.id_carga, "Cliente ausente")
        meu_tms.registrar_entrega(carga_rj.id_carga)

    if carga_cp:
        meu_tms.despachar_carga(carga_cp.id_carga)
        time.sleep(1)
        meu_tms.registrar_entrega(carga_cp.id_carga)

    if carga_bh_pesada:
        meu_tms.despachar_carga(carga_bh_pesada.id_carga)
        time.sleep(1)
        meu_tms.registrar_evento_transito(carga_bh_pesada.id_carga, "Parada para descanso do motorista.")
        time.sleep(1)
        meu_tms.registrar_entrega(carga_bh_pesada.id_carga)


    # 7. Consultar Cargas Finalizadas
    meu_tms.listar_cargas(status_filtro=CargaStatus.ENTREGUE)
    if carga_rj: meu_tms.consultar_carga(carga_rj.id_carga)

    # 8. Cancelar uma carga (Exemplo: criar uma nova e cancelar antes de despachar)
    carga_cancelar = meu_tms.criar_carga_por_skus(
        "CARGA105", end_cd_sp, end_cli_rj, [("SKU004", 1)]
    )
    if carga_cancelar:
        meu_tms.planejar_transporte(carga_cancelar.id_carga)
        meu_tms.cancelar_carga(carga_cancelar.id_carga, "Pedido duplicado pelo cliente.")
        meu_tms.consultar_carga(carga_cancelar.id_carga)

    meu_tms.listar_cargas()

ERROR:root:SKU 'SKU_NAO_EXISTE' não encontrado no catálogo.
ERROR:root:Falha ao criar item para SKU SKU_NAO_EXISTE. Carga CARGA104 não será criada.



--- Lista de Cargas ---
ID: CARGA101 | Status: CRIADA     | Orig: São Paulo       | Dest: Rio de Janeiro  | Transp: N/D                  | Custo: N/C
ID: CARGA102 | Status: CRIADA     | Orig: São Paulo       | Dest: Campinas        | Transp: N/D                  | Custo: N/C
ID: CARGA103 | Status: CRIADA     | Orig: São Paulo       | Dest: Belo Horizonte  | Transp: N/D                  | Custo: N/C
--------------------
------------------------------------------------------------
--- Carga ID: CARGA101 | Status: PLANEJADA ---
  Origem: Rua do Armazém Central, 1000 - São Paulo/SP - CEP: 01001-000
  Destino: Avenida Atlântica, 1702 - Rio de Janeiro/RJ - CEP: 22021-001
  Peso Total: 13.00 kg
  Volume Total: 0.1400 m³
  Transportadora: Log Cargas Plus
  Custo Calculado: R$ 640.00
  Previsão Entrega: 2025-05-03
  Entrega Efetiva: Pendente
  Itens:
    Item(SKU: SKU003, Qtd: 2, Peso Total: 9.00 kg, Vol Total: 0.1000 m³)
    Item(SKU: SKU004, Qtd: 5, Peso Total: 4.00 kg, Vol Total: 0.0400 m³)